# Lista de shapefiles de la clasificación de Guanajuato sentinel 2017 y 2018

Usar:

```
/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/ 
```

para 2017 y 2018 

y:

```
/LUSTRE/MADMEX/shapefiles/indi50k/indi50k_relacion_estados/Guanajuato_indi50k.csv
```

## 2017

/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k/

### 1 Encontrar lista de shapefiles de guanajuato:

csv=/LUSTRE/MADMEX/shapefiles/indi50k/indi50k_relacion_estados/Guanajuato_indi50k.csv

clasif=/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k/

```
for ind in $(sed 1d $csv);do ruta=$(find $clasif/$ind -name '*.shp');echo $ruta >> ~/lista_shapefiles_guanajuato_2017.txt;done
```

**Lo siguiente no es necesario ejecutar**

In [5]:
archivo = '/home/madmex_user/lista_shapefiles_guanajuato.txt'
#list_shapefiles = open("/home/madmex_user/lista_shapefiles_guanajuato.txt","r")

In [4]:
#list_shapefiles.close()

In [11]:
list_shapefiles=[]
with open(archivo) as src:
    list_shapefiles = src.read().splitlines()

In [13]:
list_shapefiles[0:2]

['/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k//F14C14/F14C14_madmex_sentinel2_2017.shp',
 '/LUSTRE/MADMEX/tasks/2019_tasks/mapas_steffen_thilo/sentinel_2017/indi50k//F14C13/F14C13_madmex_sentinel2_2017.shp']

#is this necessary ??:
#ogr2ogr -clipsrc /LUSTRE/MADMEX/shapefiles/shapes_estados_mexico_proyeccion_inegi_lcc/Guanajuato/Guanajuato.shp Guanajuato_sentinel_2017.shp #Guanajuato_merge.shp 

**FIN DE: Lo siguiente no es necesario ejecutar**

### 2.1 Un enfoque: 

**se requiere usar el `/home/madmex_user/shell merge_shapefiles.sh` y luego código de python (esto hay que modificar, mejor si sólo se usa python)**


```
cat merge_shapefiles.sh 
#!/bin/bash
# first parameter is list of shapefiles to merge
# second parameter is name of merge shapefile
# example: bash merge_shapefiles.sh lista_shapefiles_guanajuato.txt Guanajuato_merge

file="$2.shp"

for i in $(cat $1)
do
        if [ -f "$file" ]
        then
                echo "creating $2.shp" 
                ogr2ogr -f 'ESRI Shapefile' -update -append $file $i 
        else
                echo "merging..."
                ogr2ogr -f 'ESRI Shapefile' $file $i
        fi
done


```


Execute:


```
bash merge_shapefiles.sh lista_shapefiles_guanajuato.txt Guanajuato_merge
```

Then:

In [1]:
import os

import fiona
from fiona.crs import to_string

In [2]:
shapefile = os.path.expanduser('~') + '/Guanajuato_merge.shp'

In [3]:
shapefile

'/home/madmex_user/Guanajuato_merge.shp'

In [4]:
with fiona.open(shapefile) as src:
    schema = src.schema
    fc = [x for x in src]
    crs = to_string(src.crs)

In [5]:
schema

{'properties': OrderedDict([('id', 'int:18'),
              ('clave_31', 'int:18'),
              ('descr_31', 'str:80'),
              ('clave_17', 'int:18'),
              ('descr_17', 'str:80'),
              ('area', 'float:24.15')]),
 'geometry': 'Polygon'}

In [6]:
schema['geometry'] = ('Polygon', 'MultiPolygon')

In [7]:
filename =  os.path.expanduser('~') + '/'  + 'Guanajuato_merge' + '.gpkg'

In [8]:
filename

'/home/madmex_user/Guanajuato_merge.gpkg'

In [9]:
with fiona.open(filename, 
               'w',
               layer='Guanajuato_merge',
               schema = schema,
               crs = crs,
               driver='GPKG') as dst:
    [dst.write(x) for x in fc]

### 2.2 Otro enfoque: 

**más sencillo pero se genera un layer por cada archivo shapefile**

Execute: `/home/madmex_user/merge_shapefiles_to_geopackage.sh`

```
#!/bin/bash
# first parameter is list of shapefiles to merge
# second parameter is name of merge shapefile
# example: bash merge_shapefiles_to_geopackage.sh lista_shapefiles_guanajuato_2017.txt Guanajuato_merge

file="$2.gpkg"

for i in $(cat $1)
do
        if [ -f "$file" ]
        then
                echo "creating $2.gpkg" 
                ogr2ogr -f 'GPKG' -update -append $file $i 
        else
                echo "merging..."
                ogr2ogr -f 'GPKG' $file $i
        fi
done
```

```
bash merge_shapefiles_to_geopackage.sh lista_shapefiles_guanajuato_2017.txt Guanajuato_merge_gpkg
```

## 3 Resultados en:


1er enfoque: (2.1)

(se borró)

2o enfoque: (2.2)

```
/home/madmex_user/results/Guanajuato_sentinel2_2017.gpkg
```

O en sus rutas mapeadas a `LUSTRE`:

1er enfoque: (2.1)

(se borró)

2o enfoque: (2.2)

```
/LUSTRE/MADMEX/docker_antares3/conabio_cluster/home_madmex_user_conabio_docker_container_results/Guanajuato_sentinel2_2017.gpkg
```


## 2.3 Otra opcion:

a partir de una lista de shapefiles :

```
lista_shapefiles_guanajuato_2017.txt 
lista_shapefiles_guanajuato_2018.txt
```

se crea un solo geopackage co una sola layer sin usar ogr2ogr 

resultados en:

/home/madmex_user/results/Guanajuato_sentinel2_2017.gpkg 

/home/madmex_user/results/Guanajuato_sentinel2_2018.gpkg

In [1]:
import geopandas as gpd
import pandas as pd

In [12]:
inputfile = '/home/madmex_user/lista_shapefiles_guanajuato_2018.txt'
#inputfile = '/home/madmex_user/lista_shapefiles_chihuahua.txt'

In [13]:
with open(inputfile) as f:
    tilesfiles = f.read().splitlines()

In [14]:
len(tilesfiles)

52

In [15]:
tables = []
for tile in tilesfiles:
    tables.append(gpd.read_file(tile))

In [16]:
tables[0].tail()

,id,clave_31,descr_31,clave_17,descr_17,area,geometry
9538,9539,3,Bosque de Encino y Bosque de Galería,2,Bosques latifoliados,2.596612,"(POLYGON ((2620272.37952 1082319.65806, 262029..."
9539,9540,27,Pastizales,13,Pastizales y otra vegetación herbácea,1.162965,"(POLYGON ((2622342.37952 1082259.65806, 262238..."
9540,9541,3,Bosque de Encino y Bosque de Galería,2,Bosques latifoliados,28.477767,"(POLYGON ((2622262.37952 1082589.65806, 262230..."
9541,9542,27,Pastizales,13,Pastizales y otra vegetación herbácea,14982.584674,"(POLYGON ((2620802.37952 1099289.65806, 262081..."
9542,9543,28,Tierras Agrícolas,14,Tierras agrícolas,2.239783,"POLYGON ((2636632.37952 1082499.65806, 2636662..."


In [17]:
df = pd.concat(tables).set_index('id').reset_index().sort_index()
tables = []

In [18]:
df.tail()

,id,clave_31,descr_31,clave_17,descr_17,area,geometry
364999,5580,29,Urbano y Construido,15,Urbano y construido,1.022420,"(POLYGON ((2583022.40646 861482.6352200001, 25..."
365000,5581,29,Urbano y Construido,15,Urbano y construido,1.456670,"POLYGON ((2583242.40646 861472.6352200001, 258..."
365001,5582,29,Urbano y Construido,15,Urbano y construido,252.530344,"POLYGON ((2583012.40646 863442.6352200001, 258..."
365002,5583,28,Tierras Agrícolas,14,Tierras agrícolas,36952.938790,"(POLYGON ((2601702.40646 887412.6352200001, 26..."
365003,5584,29,Urbano y Construido,15,Urbano y construido,24.227507,"(POLYGON ((2599132.40646 862222.6352200001, 25..."


In [19]:
outputfile = '/home/madmex_user/Guanajuato_sentinel2_2018.gpkg'
#outputfile = '/home/madmex_user/Chihuahua_merge2.gpkg'

In [20]:
df.to_file(outputfile, driver='GPKG')